# COVID-19 Exploratory Data Analysis using ML.Net

## COVID-19
- As per [Wiki](https://en.wikipedia.org/wiki/Coronavirus_disease_2019) **Coronavirus disease 2019** (**COVID-19**) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease was first identified in 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.
- The virus had caused a pandemic across the globe and spreading/affecting most of the nations. 
- The purpose of notebook is to visualize the trends of virus spread in various countries and explore features present in ML.Net such as DataFrame.

### Acknowledgement
- [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data) for dataset
- [COVID-19 data visualization](https://www.kaggle.com/akshaysb/covid-19-data-visualization) by Akshay Sb

### Links

- **Dataset :** [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE - Daily reports](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports).

### Summary

Below is the summary of steps we'll be performing

1. Import necessary libraries and modules
    - Nuget packages
    - Namespaces      
     
2. Utility Functions
    - Formatters    

3. Load Dataset
    - Download Dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data)
    - Load dataset in DataFrame
    
4. Analyse Data
    - Date Range
    - display(dataframe)
    - dataframe.Head(5)
    - dataframe.Sample(6)    
    - dataframe.Description()
    - dataframe.Info()

5. Data Cleaning
    - Remove Invalid Active cases

6. Data Visualization
    - Global
        - Confirmed Vs Deaths Vs Recovered
        - Top 5 Countries with Confirmed cases
        - Top 5 Countries with Death cases
        - Top 5 Countries with Recovered cases
    - India
        - Confirmed Vs Deaths Vs Recovered

### 1. Import necessary libraries and modules

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.Data.Analysis"

//Install XPlot package
#r "nuget:XPlot.Plotly"

Installing package Microsoft.ML..................................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package Microsoft.Data.Analysis............done!

Successfully added reference to package Microsoft.Data.Analysis, version 0.3.0

Installing package XPlot.Plotly...........done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

#### Namespaces

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using XPlot.Plotly;
using Microsoft.AspNetCore.Html;
using System.IO;
using System.Net.Http;

### 2. Utility Functions

#### Formatters

In [3]:
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

### 3. Load Dataset

#### Download Dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data)

In [4]:
string lastUpdatedFile = "03-28-2020.csv";
if (!File.Exists(lastUpdatedFile))
{
    var contents = new HttpClient()
        .GetStringAsync($"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{lastUpdatedFile}").Result;
        
    File.WriteAllText(lastUpdatedFile, contents);
}

#### Load dataset in DataFrame

In [5]:
var covid19Dataframe = DataFrame.LoadCsv(lastUpdatedFile);

### 4. Analyse Data

#### Constants

In [6]:
// Column Names
const string FIPS = "FIPS";
const string ADMIN = "Admin2";
const string STATE = "Province_State";
const string COUNTRY = "Country_Region";
const string LAST_UPDATE = "Last_Update";
const string LATITUDE = "Lat";
const string LONGITUDE = "Long_";
const string CONFIRMED = "Confirmed";
const string DEATHS = "Deaths";
const string RECOVERED = "Recovered";
const string ACTIVE = "Active";
const string COMBINED_KEY = "Combined_Key";

const int TOP_COUNT = 5;
const string VALUES = "Values";
const string INDIA = "India";


In [7]:
var dateRangeDataFrame = covid19Dataframe.Columns[LAST_UPDATE].ValueCounts();
var dataRange = dateRangeDataFrame.Columns[VALUES].Sort();
var lastElementIndex = dataRange.Length - 1;

var startDate = DateTime.Parse(dataRange[0].ToString()).ToShortDateString();
var lastDate  = DateTime.Parse(dataRange[lastElementIndex].ToString()).ToShortDateString(); // Last Element

display(h4($"The data is between {startDate} and {lastDate}"));

The data is between 2/23/2020 and 3/28/2020

In [8]:
display(covid19Dataframe)

0,45001,Abbeville,South Carolina,US,2020-03-28 23:05:37,34.223335,-82.46171,3,0,0,0,"""Abbeville, South Carolina, US"""
1,22001,Acadia,Louisiana,US,2020-03-28 23:05:37,30.295065,-92.4142,9,1,0,0,"""Acadia, Louisiana, US"""
2,51001,Accomack,Virginia,US,2020-03-28 23:05:37,37.76707,-75.63235,2,0,0,0,"""Accomack, Virginia, US"""
3,16001,Ada,Idaho,US,2020-03-28 23:05:37,43.452656,-116.241554,76,0,0,0,"""Ada, Idaho, US"""
4,19001,Adair,Iowa,US,2020-03-28 23:05:37,41.330757,-94.47106,1,0,0,0,"""Adair, Iowa, US"""
5,21001,Adair,Kentucky,US,2020-03-28 23:05:37,37.1046,-85.281296,0,0,0,0,"""Adair, Kentucky, US"""
6,29001,Adair,Missouri,US,2020-03-28 23:05:37,40.190586,-92.600784,1,0,0,0,"""Adair, Missouri, US"""
7,40001,Adair,Oklahoma,US,2020-03-28 23:05:37,35.88494,-94.65859,3,0,0,0,"""Adair, Oklahoma, US"""
8,8001,Adams,Colorado,US,2020-03-28 23:05:37,39.87432,-104.33626,71,0,0,0,"""Adams, Colorado, US"""
9,16003,Adams,Idaho,US,2020-03-28 23:05:37,44.893337,-116.45452,0,0,0,0,"""Adams, Idaho, US"""
10,17001,Adams,Illinois,US,2020-03-28 23:05:37,39.988155,-91.18787,1,0,0,0,"""Adams, Illinois, US"""


In [9]:
covid19Dataframe.Head(5)

0,45001,Abbeville,South Carolina,US,2020-03-28 23:05:37,34.223335,-82.46171,3,0,0,0,"""Abbeville, South Carolina, US"""
1,22001,Acadia,Louisiana,US,2020-03-28 23:05:37,30.295065,-92.4142,9,1,0,0,"""Acadia, Louisiana, US"""
2,51001,Accomack,Virginia,US,2020-03-28 23:05:37,37.76707,-75.63235,2,0,0,0,"""Accomack, Virginia, US"""
3,16001,Ada,Idaho,US,2020-03-28 23:05:37,43.452656,-116.241554,76,0,0,0,"""Ada, Idaho, US"""
4,19001,Adair,Iowa,US,2020-03-28 23:05:37,41.330757,-94.47106,1,0,0,0,"""Adair, Iowa, US"""


In [10]:
covid19Dataframe.Sample(6)

0,6099,Stanislaus,California,US,2020-03-28 23:05:37,37.558624,-120.99707,23,0,0,0,"""Stanislaus, California, US"""
1,13181,Lincoln,Georgia,US,2020-03-28 23:05:37,33.792015,-82.45024,3,0,0,0,"""Lincoln, Georgia, US"""
2,49049,Utah,Utah,US,2020-03-28 23:05:37,40.116673,-111.665764,30,0,0,0,"""Utah, Utah, US"""
3,25009,Essex,Massachusetts,US,2020-03-28 23:05:37,42.668762,-70.94687,472,3,0,0,"""Essex, Massachusetts, US"""
4,56039,Teton,Wyoming,US,2020-03-28 23:05:37,43.935226,-110.58908,13,0,0,0,"""Teton, Wyoming, US"""
5,37155,Robeson,North Carolina,US,2020-03-28 23:05:37,34.642445,-79.10251,2,0,0,0,"""Robeson, North Carolina, US"""


In [11]:
covid19Dataframe.Description()

0,Length (excluding null values),3148,3430,3430,3430,3430,3430,3430
1,Max,99999,71.7069,178.065,92472,10023,62098,70065
2,Min,0,-42.8821,-170.132,0,0,0,-6
3,Mean,27967.148,36.889256,-82.8393,192.62566,8.936443,40.64577,107.05831


In [12]:
covid19Dataframe.Info()

0,DataType,System.Single,System.String,System.String,System.String,System.String,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.String
1,Length (excluding null values),3148,3430,3430,3430,3430,3430,3430,3430,3430,3430,3430,3430


### 5. Data Cleaning

#### Remove invalid Active cases

In [13]:
PrimitiveDataFrameColumn<bool> invalidActiveFilter = covid19Dataframe.Columns[ACTIVE].ElementwiseLessThan(0.0);
var invalidActiveDataFrame = covid19Dataframe.Filter(invalidActiveFilter);
display(invalidActiveDataFrame)

0,<null>,,Hainan,China,2020-03-24 04:29:15,19.1959,109.7453,168,6,168,-6,"""Hainan, China"""


**From above table, we could see some there were 168 confirmed and recovered cases with 6 deaths which seems invalid.
Let's remove it in next step**

In [14]:
PrimitiveDataFrameColumn<bool> activeFilter = covid19Dataframe.Columns[ACTIVE].ElementwiseGreaterThanOrEqual(0.0);
covid19Dataframe = covid19Dataframe.Filter(activeFilter);
display(covid19Dataframe.Description());

0,Length (excluding null values),3148,3429,3429,3429,3429,3429,3429
1,Max,99999,71.7069,178.065,92472,10023,62098,70065
2,Min,0,-42.8821,-170.132,0,0,0,0
3,Mean,27975.305,36.894413,-82.89547,192.63284,8.9373,40.60863,107.09128


**We have removed cases with negative active value. As per above table minimum value for Active cases is zero**

### 6. Visualization

#### Global

##### Collect Data

In [15]:
var confirmed = covid19Dataframe.Columns[CONFIRMED];
var deaths = covid19Dataframe.Columns[DEATHS];
var recovered = covid19Dataframe.Columns[RECOVERED];

var totalConfirmed = Convert.ToDouble(confirmed.Sum());
var totalDeaths = Convert.ToDouble(deaths.Sum());
var totaRecovered = Convert.ToDouble(recovered.Sum());

##### Confirmed Vs Deaths Vs Receovered cases

In [16]:
display(Chart.Plot(
    new Graph.Pie()
    {
        values = new double[]{totalConfirmed, totalDeaths, totaRecovered},
        labels = new string[] {CONFIRMED, DEATHS, RECOVERED}
    }
));

##### Top 5 Countries with Confirmed cases

In [17]:
// Get the data
var countryConfirmedGroup = covid19Dataframe.GroupBy(COUNTRY).Sum(CONFIRMED).OrderByDescending(CONFIRMED);
var topCountriesColumn = countryConfirmedGroup.Columns[COUNTRY];
var topConfirmedCasesByCountry = countryConfirmedGroup.Columns[CONFIRMED];

HashSet<string> countries = new HashSet<string>(TOP_COUNT);
HashSet<long> confirmedCases = new HashSet<long>(TOP_COUNT);
for(int index = 0; index < TOP_COUNT; index++)
{
    countries.Add(topCountriesColumn[index].ToString());
    confirmedCases.Add(Convert.ToInt64(topConfirmedCasesByCountry[index]));
}

In [18]:
var title = "Top 5 Countries : Confirmed";
var series1 = new Graph.Bar{
        x = countries.ToArray(),
        y = confirmedCases.ToArray()
    };

var chart = Chart.Plot(new []{series1});
chart.WithTitle(title);
display(chart);

##### Top 5 Countries with Deaths

In [19]:
// Get the data
var countryDeathsGroup = covid19Dataframe.GroupBy(COUNTRY).Sum(DEATHS).OrderByDescending(DEATHS);
var topCountriesColumn = countryDeathsGroup.Columns[COUNTRY];
var topDeathCasesByCountry = countryDeathsGroup.Columns[DEATHS];

HashSet<string> countries = new HashSet<string>(TOP_COUNT);
HashSet<long> deathCases = new HashSet<long>(TOP_COUNT);
for(int index = 0; index < TOP_COUNT; index++)
{
    countries.Add(topCountriesColumn[index].ToString());
    deathCases.Add(Convert.ToInt64(topDeathCasesByCountry[index]));
}

In [20]:
var title = "Top 5 Countries : Deaths";
var series1 = new Graph.Bar{
        x = countries.ToArray(),
        y = deathCases.ToArray()
    };

var chart = Chart.Plot(new []{series1});
chart.WithTitle(title);
display(chart);

##### Top 5 Countries with Recovered cases

In [21]:
// Get the data
var countryRecoveredGroup = covid19Dataframe.GroupBy(COUNTRY).Sum(RECOVERED).OrderByDescending(RECOVERED);
var topCountriesColumn = countryRecoveredGroup.Columns[COUNTRY];
var topRecoveredCasesByCountry = countryRecoveredGroup.Columns[RECOVERED];

HashSet<string> countries = new HashSet<string>(TOP_COUNT);
HashSet<long> recoveredCases = new HashSet<long>(TOP_COUNT);
for(int index = 0; index < TOP_COUNT; index++)
{
    countries.Add(topCountriesColumn[index].ToString());
    recoveredCases.Add(Convert.ToInt64(topRecoveredCasesByCountry[index]));
}

In [22]:
var title = "Top 5 Countries : Recovered";
var series1 = new Graph.Bar{
        x = countries.ToArray(),
        y = recoveredCases.ToArray()
    };

var chart = Chart.Plot(new []{series1});
chart.WithTitle(title);
display(chart);

#### India

##### Collect Data

##### Confirmed Vs Deaths Vs Receovered cases

In [23]:
PrimitiveDataFrameColumn<bool> indiaFilter = covid19Dataframe.Columns[COUNTRY].ElementwiseEquals(INDIA);
var indiaDataFrame = covid19Dataframe.Filter(indiaFilter);
display(indiaDataFrame.Head((int)indiaDataFrame.Rows.Count));
            
var indiaConfirmed = indiaDataFrame.Columns[CONFIRMED];
var indiaDeaths = indiaDataFrame.Columns[DEATHS];
var indiaRecovered = indiaDataFrame.Columns[RECOVERED];

var indiaTotalConfirmed = Convert.ToDouble(indiaConfirmed.Sum());
var indiaTotalDeaths = Convert.ToDouble(indiaDeaths.Sum());
var indiaTotaRecovered = Convert.ToDouble(indiaRecovered.Sum());

0,<null>,,,India,2020-03-28 23:05:25,20.593683,78.96288,987,24,84,879,India


In [24]:
display(Chart.Plot(
    new Graph.Pie()
    {
        values = new double[]{indiaTotalConfirmed, indiaTotalDeaths, indiaTotaRecovered},
        labels = new string[] {CONFIRMED, DEATHS, RECOVERED}
    }
));

## References
- [Using ML.NET in Jupyter notebooks](https://devblogs.microsoft.com/cesardelatorre/using-ml-net-in-jupyter-notebooks/)
- [An Introduction to DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/)
- [DataFrame - Sample](https://github.com/dotnet/interactive/blob/master/NotebookExamples/csharp/Samples/HousingML.ipynb)
- [Getting started with ML.NET in Jupyter Notebooks](https://xamlbrewer.wordpress.com/2020/02/20/getting-started-with-ml-net-in-jupyter-notebooks/)
- [Tips and tricks for C# Jupyter notebook](https://ewinnington.github.io/posts/jupyter-tips-csharp)

#  ******************** Be Safe **********************